In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

queries = {
    "Overall_Churn": "SELECT ROUND(AVG(churn::numeric)*100,2) AS churn_rate_percent FROM telco_churn;",
    "Churn_By_Contract": """
        SELECT contract, ROUND(AVG(churn::numeric)*100,2) AS churn_rate_percent
        FROM telco_churn
        GROUP BY contract
        ORDER BY churn_rate_percent DESC;
    """,
    "Churn_By_InternetService": """
        SELECT internetservice, ROUND(AVG(churn::numeric)*100,2) AS churn_rate_percent
        FROM telco_churn
        GROUP BY internetservice
        ORDER BY churn_rate_percent DESC;
    """,
    "Churn_By_PaymentMethod": """
        SELECT paymentmethod, ROUND(AVG(churn::numeric)*100,2) AS churn_rate_percent
        FROM telco_churn
        GROUP BY paymentmethod
        ORDER BY churn_rate_percent DESC;
    """,
    "Churn_By_TenureGroup": """
        SELECT 
            CASE 
                WHEN tenure < 12 THEN '0–1 year'
                WHEN tenure BETWEEN 12 AND 24 THEN '1–2 years'
                WHEN tenure BETWEEN 25 AND 48 THEN '2–4 years'
                ELSE '4+ years'
            END AS tenure_group,
            ROUND(AVG(churn::numeric)*100,2) AS churn_rate_percent
        FROM telco_churn
        GROUP BY tenure_group
        ORDER BY churn_rate_percent DESC;
    """
}

# Export each query result to a separate CSV file
os.makedirs("../excel", exist_ok=True)
for name, query in queries.items():
    df = pd.read_sql(query, engine)
    path = f"../excel/{name}.csv"
    df.to_csv(path, index=False)
    print(f"✅ Exported: {path}")


✅ Exported: ../excel/Overall_Churn.csv
✅ Exported: ../excel/Churn_By_Contract.csv
✅ Exported: ../excel/Churn_By_InternetService.csv
✅ Exported: ../excel/Churn_By_PaymentMethod.csv
✅ Exported: ../excel/Churn_By_TenureGroup.csv
